<a href="https://colab.research.google.com/github/Sylvester107/Few-Shot-Learning-for-Effective-Categorization-and-Recommendation-of-Low-Frequency-Products/blob/main/paper_1304_first_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
#!pip install transformers
#!pip install sentence_transformers
#import torch
#from transformers import AutoTokenizer, AutoModel
import pickle

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Few shot learning for categorization and recommendation/training_data_uk_only_labeled.csv")
data.head()

,product,label,title,price,brand,color,size,model,material,author,desc
0,B0BFDL54Y7,1,Wokii Vegetable Chopper Vegetable Slicer 14 in...,15.99,Wokii,Blue,33x12x12cm/12.87*4.68*4.68inches,NaN,Acrylonitrile Butadi,NaN,"【Easy To Clean】The blade is easy to remove, ea..."
1,B0BFDR9X13,0,Wokii Vegetable Chopper Vegetable Slicer 14 in...,14.99,Wokii,Grey+green,33x12x12cm/12.87*4.68*4.68inches,NaN,Acrylonitrile Butadi,NaN,"【Easy To Clean】The blade is easy to remove, ea..."
2,B07J4WF8VH,1,Vinsani® Kitchen Vegetable Cutter Onion Garlic...,16.99,Vinsani,White,NaN,NaN,Stainless Steel,NaN,CAUTION: If the vegetable or fruit has a tough...
3,B07Y21LDJX,0,"ADOV Vegetable Chopper, 14 in 1 Multi-Function...",21.99,ADOV,Blue,NaN,NaN,"Premium ABS Plastic, Stainless Steel",NaN,【EASY CLEANING】 ADOV vegetable slicer can be c...
4,B07Y227WNJ,0,"ADOV Vegetable Chopper, 14 in 1 Multi-Function...",21.99,ADOV,Grey,NaN,NaN,"Premium ABS Plastic, Stainless Steel",NaN,【EASY CLEANING】 ADOV vegetable slicer can be c...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494409 entries, 0 to 494408
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   product   494409 non-null  object 
 1   label     494409 non-null  int64  
 2   title     494409 non-null  object 
 3   price     494409 non-null  float64
 4   brand     490162 non-null  object 
 5   color     374047 non-null  object 
 6   size      298149 non-null  object 
 7   model     240793 non-null  object 
 8   material  295942 non-null  object 
 9   author    18582 non-null   object 
 10  desc      455736 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 41.5+ MB


In [ ]:
data.isnull().sum()

,0
product,0
label,0
title,0
price,0
brand,4247
color,120362
size,196260
model,253616
material,198467
author,475827


store the Text part of the data for and generate their embeddings
0. Replace NAN with defaults#had tomeback to make this change as float was not subsriptable and error during embedding generation
1. the author column has too many NAN values so it won't be included
2. Product is unique so it won't be included

In [ ]:
#for each column if the value at row i is nan set to column name
for col in data.columns:
  data.fillna(col, inplace=True)

In [ ]:
thousand_samples = data.sample(n=3000, random_state=42)

In [ ]:
training_data_text = thousand_samples[['title','brand', 'color', 'size', 'desc', 'material', 'model']]
training_data_text.head()

,title,brand,color,size,desc,material,model
448226,WERNNSAI Flamingo Party Invitations with Envel...,WERNNSAI,Flamingo Pink,product,🍍【DETAILS】 - We provide adhesive tape on the b...,Paper,WR-Flamingo Invitation-001-UK
350110,BENECREAT 30 Packs Brown Kraft Paper Boxes Sma...,BENECREAT,Brown,8.5x6x3cm,"[Good Quality]: Made of sturdy paper, environm...",Plastic,product
420407,Border Terrier Calendar 2023 | Square Dog Bree...,Avonside Publishing Ltd,product,product,product,product,product
260449,"Unisex 4 LED Lighted Beanie Cap, USB Rechargea...",WEITOO,Blue,One size,Best Christmas Gift-Bought for My Husband Our ...,Acrylic,product
396101,"REDKEN Colour Extend Magnetics Shampoo, 1000 ml",REDKEN,product,1 l (Pack of 1),Hair type: Color Treated,product,0884486136336


In [ ]:
training_data_text.isnull().sum()

,0
title,0
brand,0
color,0
size,0
desc,0
material,0
model,0


We will create the first supervised model not few-shot based

1. Generate model embeddings using transformer
2. Train classifier(XGboost, GRADBoost etc)

I am reducing the training data once again to test method feasibility, as the ram provided by colab is too small



In [ ]:
#training_data_text = text_data.sample(n=1000, random_state=42)

In [ ]:
#import sentence_transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# for each of the column in data_text generate an embedding add to a dictionary
text_embeddings = {}

for col in training_data_text.columns:
  text_embeddings[col] = model.encode(training_data_text[col].values,show_progress_bar =True)
  text_embeddings[col] = np.array(text_embeddings[col])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

1. convert text embeddings to a dataframe
2. add price and label to this dataframe
3. then split into train and test
4. after that train a classifier

NB: we are not performing a fine-tuning because we don't have a pre-trained  model ready


In [ ]:
#convert text embedding to a pickle file for retrieval in case of crash
#with open('text_embeddings.pkl', 'wb') as f:
 #   pickle.dump(text_embeddings, f)

In [ ]:
text_embeddings['brand'][0]

array([-9.47333649e-02,  1.61780007e-02, -1.23976804e-01,  6.08473308e-02,
       -2.63606906e-02, -1.56721435e-02,  8.96072090e-02,  2.04411279e-02,
        5.70392236e-02, -2.75329705e-02,  1.43329119e-02, -7.73995072e-02,
       -7.31678829e-02,  6.64781779e-02, -2.56281383e-02, -4.26454470e-02,
        2.25475966e-03,  2.90483814e-02, -4.68797423e-02, -7.94182196e-02,
       -7.18546659e-02,  6.05011024e-02,  3.07192802e-02,  1.99930314e-02,
        4.07910421e-02, -1.62909813e-02, -1.05812000e-02,  2.07616948e-02,
        6.65747887e-03, -6.72302693e-02,  5.77732772e-02, -2.23878212e-02,
        3.11963297e-02, -1.99739411e-02, -2.58087292e-02,  4.26683314e-02,
       -3.89305018e-02,  1.42248617e-02,  2.36931257e-02, -2.93980842e-03,
       -4.25087176e-02, -3.41552380e-03,  1.77468397e-02,  1.32267922e-02,
       -2.06489824e-02,  2.95582879e-02, -1.26432791e-01,  7.84012899e-02,
        6.83402941e-02,  7.27718771e-02, -3.56775559e-02,  4.27131951e-02,
        1.81364175e-02, -

In [ ]:
# append price and labels in the text_data to text_embeddings
text_embeddings['price'] = thousand_samples['price'].values
#text_embeddings['label'] = data['label'].values #not necessary


NameError: name 'text_embeddings' is not defined

In [ ]:
#save text embeddings to pickle to drive
with open('/content/drive/MyDrive/Few shot learning for categorization and recommendation/text_embeddings.pkl', 'wb') as f:
    pickle.dump(text_embeddings, f)

Part 2, change runtime to cpu and run a few lines of code above and continue from here, don't run transformer part

In [ ]:
#load pickle file
with open('/content/drive/MyDrive/Few shot learning for categorization and recommendation/text_embeddings.pkl', 'rb') as f:
    text_embeddings = pickle.load(f)


In [ ]:
embedding_list = list(text_embeddings.values())[:-1]
embedding_list.append(text_embeddings['price'].reshape(-1, 1))  # Reshape price to 2D array
feature_matrix = np.concatenate(embedding_list, axis=1)  # Concatenate along columns (axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, thousand_samples['label'].values, test_size=0.2, random_state=42)


Next we train all 3 boosting models and evaluate them

In [ ]:
Gradboost = GradientBoostingClassifier(n_estimators=100,random_state=0)
Gradboost.fit(X_train, y_train)
Gradboost.score(X_test, y_test)

0.795

In [ ]:
Xgboost = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
Xgboost.fit(X_train, y_train)
Xgboost.score(X_test, y_test)

0.805

In [ ]:
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
ada.fit(X_train, y_train)
ada.score(X_test, y_test)

0.7783333333333333

In [ ]:
# try svc
from sklearn.svm import SVC
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.8066666666666666

SVC learn at a faster rate with in less time

1. Use gridsearch to improve performance of both SVC and XGB
2. for each of these models pass all the train data into the model, no spliting into test/train then make predictions take the prediction probabilities and append to the training data again for few-shot ranking.

In [ ]:
# find best params for svc using gridsearch
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']}

grid= GridSearchCV(SVC(),param_grid,refit=False,verbose=2)
grid.fit(X_train,y_train)

best_params = grid.best_params_
print(best_params)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   9.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   9.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.1s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   9.7s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.1s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   9.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.2s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   8.3s
[CV] END ......................C=0.1, gamma=0.01

AttributeError: This 'SVC' has no attribute 'predict_proba'

In [ ]:
#{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
#retrain svc with best params, pass the whole training data i.e feature matrix
svc = SVC(kernel='rbf', C=0.1, gamma=1,probability=True)
svc.fit(feature_matrix, thousand_samples['label'].values)

#generate and store the ouput probabilites of each sample in a list
output_probabilities = svc.predict_proba(feature_matrix)
print(output_probabilities)

[[0.09293491 0.90706509]
 [0.0933766  0.9066234 ]
 [0.09289707 0.90710293]
 ...
 [0.09362302 0.90637698]
 [0.09390178 0.90609822]
 [0.09188672 0.90811328]]


In [ ]:

# attach output probability of each sample as a new column in feature matrix
for i in range(output_probabilities.shape[1]):
    feature_matrix = np.hstack((feature_matrix, output_probabilities[:, i].reshape(-1, 1)))



In [ ]:
feature_matrix.shape

(3000, 2691)

The last value in our feature matrix is the probability that this product is a low frequency product, learned using SVC

In [ ]:
feature_matrix[1][-1]

0.9066234004616708

In [ ]:
#save feature matrix to drive using pickle
with open('/content/drive/MyDrive/Few shot learning for categorization and recommendation/feature_matrix.pkl', 'wb') as f:
    pickle.dump(feature_matrix, f)

XGboost seems to be performing very well, focus on optimizing it

techniques:
- Gridsearch for hyperparameter tuning
- add 1000 more data points


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150], 'grow_policy' : ['depthwise','lossguide'], 'learning_rate': [0.01, 0.1, 1], }




MRR@100

In [ ]:
"""import numpy as np
from scipy.stats import rankdata

def mean_reciprocal_rank(y_true, y_pred, k=100):
    mrrs = []
    for true_labels, pred_scores in zip(y_true, y_pred):
        ranks = rankdata(-pred_scores, method='ordinal')
        true_ranks = ranks[np.argmax(true_labels)]
        if true_ranks <= k:
            mrrs.append(1.0 / true_ranks)
        else:
            mrrs.append(0)
    return np.mean(mrrs) """